In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)

In [4]:
df = spark.read.csv("dataset/conditions.csv").limit(20)

NameError: name 'spark' is not defined

In [3]:
Conditions =pd.read_csv("dataset/conditions.csv")
sc = spark.sparkContext

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [3]:
Conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2017-01-14,2017-03-30,09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a,88e540ab-a7d7-47de-93c1-720a06f3d601,65363002,Otitis media
1,2012-09-15,2012-09-16,b0a03e8c-8d0f-4242-9548-40f4d294eba8,e89414dc-d0c6-478f-86c0-d08bac6ad0a2,241929008,Acute allergic reaction
2,2018-06-17,2018-06-24,09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a,c14325b0-f7ec-4314-bba8-dddc37f0067d,444814009,Viral sinusitis (disorder)
3,2019-04-19,2019-09-26,09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a,71af18ee-3157-408b-b576-e96445664c0a,65363002,Otitis media
4,2019-04-27,2019-05-18,09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a,411d4eae-72d1-4786-a28e-3b744cf17a47,444814009,Viral sinusitis (disorder)


In [7]:
sparkDF=spark.createDataFrame(Conditions.astype(str)) 
sparkDF.printSchema()
sparkDF.show()

KeyboardInterrupt: 